In [42]:
!pip install linearmodels
!pip install openpyxl --upgrade openpyxl

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import t, norm
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import linearmodels as lm
from linearmodels.panel import PanelOLS

In [44]:
file_path1 = '/content/cap iq.xlsx'
data_wide = pd.read_excel(file_path1, sheet_name = 0)

In [45]:
data_wide.head()

,Company Name,Exchange:Ticker,Tickers only,Industry Classifications,Industry Classifications.1,Price Close - Quarter -19,Price Close - Quarter -18,Price Close - Quarter -17,Price Close - Quarter -16,Price Close - Quarter -15,...,"Free Cash Flow Actual - Capital IQ [FQ-10] ($USDmm, Historical rate)","Free Cash Flow Actual - Capital IQ [FQ-9] ($USDmm, Historical rate)","Free Cash Flow Actual - Capital IQ [FQ-8] ($USDmm, Historical rate)","Free Cash Flow Actual - Capital IQ [FQ-7] ($USDmm, Historical rate)","Free Cash Flow Actual - Capital IQ [FQ-6] ($USDmm, Historical rate)","Free Cash Flow Actual - Capital IQ [FQ-5] ($USDmm, Historical rate)","Free Cash Flow Actual - Capital IQ [FQ-4] ($USDmm, Historical rate)","Free Cash Flow Actual - Capital IQ [FQ-3] ($USDmm, Historical rate)","Free Cash Flow Actual - Capital IQ [FQ-2] ($USDmm, Historical rate)","Free Cash Flow Actual - Capital IQ [FQ-1] ($USDmm, Historical rate)"
0,Apple Inc. (NasdaqGS:AAPL),NasdaqGS:AAPL,AAPL,Information Technology (Primary),Information Technology,293.65,254.29,364.80,115.81,132.69,...,25652.0,20790.0,20838.0,30218.0,25644.0,24287.0,19435.0,37503.0,20694.0,26707.0
1,Abbott Laboratories (NYSE:ABT),NYSE:ABT,ABT,Health Care (Primary),Health Care,78.91,91.43,108.83,109.49,119.84,...,1744.0,2031.0,2313.0,1716.0,763.0,696.0,1317.0,2283.0,627.0,1427.0
2,Accenture plc (NYSE:ACN),NYSE:ACN,ACN,Information Technology (Primary),Information Technology,201.16,180.59,201.62,239.93,249.09,...,1990.8,2870.0,3613.1,396.6,2222.6,3147.9,3229.1,429.6,1991.2,3017.9
3,Adobe Inc. (NasdaqGS:ADBE),NasdaqGS:ADBE,ADBE,Information Technology (Primary),Information Technology,345.12,386.60,513.39,478.47,459.67,...,1669.0,1914.0,1579.0,2234.0,1592.0,2018.0,1782.0,1550.0,1137.0,1899.0
4,"Analog Devices, Inc. (NasdaqGS:ADI)",NasdaqGS:ADI,ADI,Information Technology (Primary),Information Technology,109.75,109.60,114.85,118.53,147.33,...,1103.0,1083.0,844.8,1230.1,797.2,817.9,710.9,916.0,619.7,701.1


In [46]:
data_wide.columns

Index(['Company Name', 'Exchange:Ticker', 'Tickers only',
       'Industry Classifications', 'Industry Classifications.1',
       'Price Close - Quarter -19', 'Price Close - Quarter -18',
       'Price Close - Quarter -17', 'Price Close - Quarter -16',
       'Price Close - Quarter -15', 'Price Close - Quarter -14',
       'Price Close - Quarter -13', 'Price Close - Quarter -12',
       'Price Close - Quarter -11', 'Price Close - Quarter -10',
       'Price Close - Quarter -9', 'Price Close - Quarter -8',
       'Price Close - Quarter -7', 'Price Close - Quarter -6',
       'Price Close - Quarter -5', 'Price Close - Quarter -4',
       'Price Close - Quarter -3', 'Price Close - Quarter -2',
       'Price Close - Quarter -1', 'EPS FQ -19', 'EPS FQ -18', 'EPS FQ -17',
       'EPS FQ -16', 'EPS FQ -15', 'EPS FQ -14', 'EPS FQ -13', 'EPS FQ -12',
       'EPS FQ -11', 'EPS FQ -10', 'EPS FQ -9', 'EPS FQ -8', 'EPS FQ -7',
       'EPS FQ -6', 'EPS FQ -5', 'EPS FQ -4', 'EPS FQ -3', 'EPS FQ -2',


In [47]:
import pandas as pd

# Melt the DataFrame to make it long
long_df = pd.melt(
    data_wide,
    id_vars=['Company Name', 'Exchange:Ticker', 'Tickers only',
       'Industry Classifications', 'Industry Classifications.1'],  # Keep these columns as-is
    var_name='Metric_Quarters',                  # Combine columns into one
    value_name='Value'                       # The corresponding values
)


In [48]:
long_df.head()

,Company Name,Exchange:Ticker,Tickers only,Industry Classifications,Industry Classifications.1,Metric_Quarters,Value
0,Apple Inc. (NasdaqGS:AAPL),NasdaqGS:AAPL,AAPL,Information Technology (Primary),Information Technology,Price Close - Quarter -19,293.65
1,Abbott Laboratories (NYSE:ABT),NYSE:ABT,ABT,Health Care (Primary),Health Care,Price Close - Quarter -19,78.91
2,Accenture plc (NYSE:ACN),NYSE:ACN,ACN,Information Technology (Primary),Information Technology,Price Close - Quarter -19,201.16
3,Adobe Inc. (NasdaqGS:ADBE),NasdaqGS:ADBE,ADBE,Information Technology (Primary),Information Technology,Price Close - Quarter -19,345.12
4,"Analog Devices, Inc. (NasdaqGS:ADI)",NasdaqGS:ADI,ADI,Information Technology (Primary),Information Technology,Price Close - Quarter -19,109.75


In [49]:
long_df['Metric_Quarters'].unique()

array(['Price Close - Quarter -19', 'Price Close - Quarter -18',
       'Price Close - Quarter -17', 'Price Close - Quarter -16',
       'Price Close - Quarter -15', 'Price Close - Quarter -14',
       'Price Close - Quarter -13', 'Price Close - Quarter -12',
       'Price Close - Quarter -11', 'Price Close - Quarter -10',
       'Price Close - Quarter -9', 'Price Close - Quarter -8',
       'Price Close - Quarter -7', 'Price Close - Quarter -6',
       'Price Close - Quarter -5', 'Price Close - Quarter -4',
       'Price Close - Quarter -3', 'Price Close - Quarter -2',
       'Price Close - Quarter -1', 'EPS FQ -19', 'EPS FQ -18',
       'EPS FQ -17', 'EPS FQ -16', 'EPS FQ -15', 'EPS FQ -14',
       'EPS FQ -13', 'EPS FQ -12', 'EPS FQ -11', 'EPS FQ -10',
       'EPS FQ -9', 'EPS FQ -8', 'EPS FQ -7', 'EPS FQ -6', 'EPS FQ -5',
       'EPS FQ -4', 'EPS FQ -3', 'EPS FQ -2', 'EPS FQ -1',
       'EPS Actual - Capital IQ [FQ-19] ($USD, Historical rate)',
       'EPS Actual - Capital IQ [FQ-1

In [50]:
long_df['Metric'] = long_df['Metric_Quarters'].str.extract(r'^(.*?) -')  # Extract before " - "
long_df['Period'] = long_df['Metric_Quarters'].str.extract(r'-(\d{1,2})')

# Clean up other details (if any specific pattern exists)
long_df['Metric'] = long_df['Metric'].str.strip()  # Clean extra spaces


In [51]:
long_df.head()

,Company Name,Exchange:Ticker,Tickers only,Industry Classifications,Industry Classifications.1,Metric_Quarters,Value,Metric,Period
0,Apple Inc. (NasdaqGS:AAPL),NasdaqGS:AAPL,AAPL,Information Technology (Primary),Information Technology,Price Close - Quarter -19,293.65,Price Close,19
1,Abbott Laboratories (NYSE:ABT),NYSE:ABT,ABT,Health Care (Primary),Health Care,Price Close - Quarter -19,78.91,Price Close,19
2,Accenture plc (NYSE:ACN),NYSE:ACN,ACN,Information Technology (Primary),Information Technology,Price Close - Quarter -19,201.16,Price Close,19
3,Adobe Inc. (NasdaqGS:ADBE),NasdaqGS:ADBE,ADBE,Information Technology (Primary),Information Technology,Price Close - Quarter -19,345.12,Price Close,19
4,"Analog Devices, Inc. (NasdaqGS:ADI)",NasdaqGS:ADI,ADI,Information Technology (Primary),Information Technology,Price Close - Quarter -19,109.75,Price Close,19


In [52]:
long_df['Metric'].unique()

array(['Price Close', 'EPS FQ', 'EPS Actual', 'DPS Actual',
       'Free Cash Flow Actual'], dtype=object)

In [53]:
final_df = long_df.pivot_table(
    index=['Company Name', 'Exchange:Ticker', 'Tickers only',
       'Industry Classifications', 'Industry Classifications.1','Period'],  # Grouping columns
    columns='Metric',                              # Columns to pivot
    values='Value'                                 # Values for the pivot
).reset_index()


In [54]:
final_df.tail()

Metric,Company Name,Exchange:Ticker,Tickers only,Industry Classifications,Industry Classifications.1,Period,DPS Actual,EPS Actual,EPS FQ,Free Cash Flow Actual,Price Close
1097,"Zimmer Biomet Holdings, Inc. (NYSE:ZBH)",NYSE:ZBH,ZBH,Health Care (Primary),Health Care,5,0.24,1.82,1.81,165.1,112.22
1098,"Zimmer Biomet Holdings, Inc. (NYSE:ZBH)",NYSE:ZBH,ZBH,Health Care (Primary),Health Care,6,0.24,1.89,1.64,178.2,145.60
1099,"Zimmer Biomet Holdings, Inc. (NYSE:ZBH)",NYSE:ZBH,ZBH,Health Care (Primary),Health Care,7,0.24,1.88,1.83,114.7,129.20
1100,"Zimmer Biomet Holdings, Inc. (NYSE:ZBH)",NYSE:ZBH,ZBH,Health Care (Primary),Health Care,8,0.24,1.58,1.56,332.0,127.50
1101,"Zimmer Biomet Holdings, Inc. (NYSE:ZBH)",NYSE:ZBH,ZBH,Health Care (Primary),Health Care,9,0.24,1.82,1.64,240.4,104.55


In [55]:
final_df.rename(columns={'EPS FQ': 'EPS Forecast', 'Period': 'FQ -'}, inplace=True)

In [56]:
final_df

Metric,Company Name,Exchange:Ticker,Tickers only,Industry Classifications,Industry Classifications.1,FQ -,DPS Actual,EPS Actual,EPS Forecast,Free Cash Flow Actual,Price Close
0,3M Company (NYSE:MMM),NYSE:MMM,MMM,Industrials (Primary),Industrials,1,0.70,1.93,1.68,752.0,248.50
1,3M Company (NYSE:MMM),NYSE:MMM,MMM,Industrials (Primary),Industrials,10,1.49,2.65,2.32,587.0,129.41
2,3M Company (NYSE:MMM),NYSE:MMM,MMM,Industrials (Primary),Industrials,11,1.48,2.31,2.02,1449.0,148.88
3,3M Company (NYSE:MMM),NYSE:MMM,MMM,Industrials (Primary),Industrials,12,1.48,2.45,2.21,1531.0,177.63
4,3M Company (NYSE:MMM),NYSE:MMM,MMM,Industrials (Primary),Industrials,13,1.47,2.59,2.28,1493.0,175.42
...,...,...,...,...,...,...,...,...,...,...,...
1097,"Zimmer Biomet Holdings, Inc. (NYSE:ZBH)",NYSE:ZBH,ZBH,Health Care (Primary),Health Care,5,0.24,1.82,1.81,165.1,112.22
1098,"Zimmer Biomet Holdings, Inc. (NYSE:ZBH)",NYSE:ZBH,ZBH,Health Care (Primary),Health Care,6,0.24,1.89,1.64,178.2,145.60
1099,"Zimmer Biomet Holdings, Inc. (NYSE:ZBH)",NYSE:ZBH,ZBH,Health Care (Primary),Health Care,7,0.24,1.88,1.83,114.7,129.20
1100,"Zimmer Biomet Holdings, Inc. (NYSE:ZBH)",NYSE:ZBH,ZBH,Health Care (Primary),Health Care,8,0.24,1.58,1.56,332.0,127.50


In [57]:
final_df.value_counts('FQ -')

,count
FQ -,
1,58
19,58
8,58
7,58
6,58
5,58
4,58
3,58
2,58


In [58]:
final_df.value_counts('Company Name')

,count
Company Name,
3M Company (NYSE:MMM),19
QUALCOMM Incorporated (NasdaqGS:QCOM),19
Lam Research Corporation (NasdaqGS:LRCX),19
Lockheed Martin Corporation (NYSE:LMT),19
Masco Corporation (NYSE:MAS),19
Medtronic plc (NYSE:MDT),19
Microchip Technology Incorporated (NasdaqGS:MCHP),19
Microsoft Corporation (NasdaqGS:MSFT),19
"NetApp, Inc. (NasdaqGS:NTAP)",19


In [59]:
final_df['FQ -'] = pd.to_numeric(final_df['FQ -'])
final_df.sort_values(by=['Company Name', 'FQ -'], inplace=True)
final_df.to_excel('final_df.xlsx', index=False)